In [151]:
import pandas as pd 

tr = 1 
preds_sf = f"./predictions/match_preds/t{tr}_euros2024.csv"

preds_df = pd.read_csv(preds_sf)
preds_df = preds_df.drop(["Unnamed: 1"], axis=1)
preds_df = preds_df.iloc[:35,]


In [152]:
preds_df.head()

,stage,team_a,res_a,res_b,team_b,date
0,Group A,Germany,W,L,Scotland,2024-06-14
1,Group A,Hungary,L,W,Switzerland,2024-06-15
2,Group A,Germany,D,D,Hungary,2024-06-19
3,Group A,Scotland,L,W,Switzerland,2024-06-19
4,Group A,Switzerland,D,D,Germany,2024-06-23


In [153]:
import requests 
from bs4 import BeautifulSoup

results = "https://terrikon.com/en/euro-2024"
res_page = requests.get(results)
res_text = res_page.text 
res_soup = BeautifulSoup(res_text)

In [154]:
res_tbls = res_soup.find_all("table", class_="gameresult")
res_tbls = res_tbls[4:]

In [155]:
import io 

res_dfs = [pd.concat(pd.read_html(io.StringIO(table.prettify()))) for table in res_tbls]
# concat needed as without, each index was a list 

In [156]:
def fix_table(tbl, grp):
    res_opps = {"W": "L", 
                "D": "D", 
                "L": "W",
                "err": "err"
               }
    
    stg = [f"Group {grp}" for i in range(tbl.shape[0])]
    tbl[0] = stg
    tbl[4] = tbl[3]

    s_lines = tbl[2]
    for i in range(len(s_lines)): 
        scoreline = s_lines[i].split(":")
        scr_a = scoreline[0]
        scr_b = scoreline[1] 

        res_a = "err"
        if scr_a > scr_b: 
            res_a = "W"
        elif scr_a == scr_b:
            res_a = "D"
        elif scr_a < scr_b: 
            res_a = "L"
        res_b = res_opps[res_a]
        tbl.iloc[i, 2] = res_a 
        tbl.iloc[i, 3] = res_b
    
    new_cols = ["stage", "team_a", "res_a", "res_b", "team_b", "date"]
    tbl.columns = new_cols

    tbl['date'] = pd.to_datetime(tbl['date'], format="%d.%m.%y")    
                  

In [157]:
grps = ["A", "B", "C", "D", "E", "F"] 
for r in range(len(res_dfs)): 
    fix_table(res_dfs[r], grps[r])

In [158]:
res_dfs[0]

,stage,team_a,res_a,res_b,team_b,date
0,Group A,Germany,W,L,Scotland,2024-06-14
1,Group A,Hungary,L,W,Switzerland,2024-06-15
2,Group A,Germany,W,L,Hungary,2024-06-19
3,Group A,Scotland,D,D,Switzerland,2024-06-19
4,Group A,Switzerland,D,D,Germany,2024-06-23
5,Group A,Scotland,L,W,Hungary,2024-06-23
